In [1]:
from utils import kappa, ode_system, dl_func, ratio, fs8, chi_squared

In [2]:
import numpy as np

import emcee

from scipy.optimize import minimize
from scipy.integrate import solve_ivp

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

In [3]:
om_fdl = [0.3, 0.3, 0.266, 0.3, 0.31, 0.3, 0.27, 0.27, 0.25, 0.25,
          0.274,0.307115, 0.27, 0.27, 0.27, 0.3, 0.3, 0.27]

file_path = 'data/Cij_WiggleZ.txt'
Wiggle = np.loadtxt(file_path)

data_file_path = 'data/data_growth_2016_main.txt'
data = np.loadtxt(data_file_path)

In [4]:
z_data = data[:,0]
y_data = data[:,1]
sigma = data[:,2]

cov = np.eye(len(z_data))

# Replace the diagonal of the identity matrix with the array elements
np.fill_diagonal(cov, sigma**2)
cov[12:15, 12:15] = Wiggle
cov_inv = np.linalg.inv(cov)

a_data = 1/(z_data+1)

a_data = a_data[::-1]
y_data = y_data[::-1]
sigma  = sigma[::-1]
om_fdl = om_fdl[::-1]

In [5]:

ai,af,da = 1e-3,1, 1e-4
yi,xi = ai,1
N = int(round((af-ai)/da))
a_values = np.linspace(ai, af, N)
indices = np.searchsorted(a_values, a_data, side="left")
a_values[indices] = a_data
initial_conditions = [yi, xi]

initial_guess = [ 0.3, 0.8]
nll = lambda *args: chi_squared(*args)

result_L = minimize(nll, initial_guess, args = (fs8, a_values, a_data, y_data, cov_inv, om_fdl, "LCDM"), method='Nelder-Mead')
result_G = minimize(nll, initial_guess, args = (fs8, a_values, a_data, y_data, cov_inv, om_fdl, "GCDM"), method='Nelder-Mead')


optimal_params_L = result_L.x
optimal_params_G = result_G.x

print('LCDM = ', optimal_params_L, result_L.message, result_L.success)
print('GCDM =',  optimal_params_G, result_G.message, result_G.success)

LCDM =  [0.20732382 0.88652202] Optimization terminated successfully. True
GCDM = [0.24172271 0.85914095] Optimization terminated successfully. True


In [6]:
def lnprior(theta,n):
  if n == 'LCDM':
    Om, sig = theta
    if 0< Om < 1  and 0<= sig <= 2:
      return 0.
    else:
      return -np.inf
  if n == 'GCDM':
    b1, sig = theta
    ones = np.ones(N)
    if -2< b1 < 2  and 0<= sig <= 2 and (kappa(a_values,0,b1)>=ones).all():
      return 0.
    else:
       return -np.inf
  

def lnprob(theta, model, a_values, a_data, y_data, cov_inv, om_fdl,n):
    lp = lnprior(theta,n)
    if not np.isfinite(lp):
        return -np.inf
    return lp - chi_squared(theta, model, a_values, a_data, y_data, cov_inv, om_fdl,n)

runs = 3000

nwalkers = 30
pos_L = [optimal_params_L + 1e-3*np.random.randn(2) for i in range(nwalkers)]
pos_G = [optimal_params_G + 1e-3*np.random.randn(2) for i in range(nwalkers)]

In [7]:
sampler_L = emcee.EnsembleSampler(nwalkers, 2, lnprob, args=(fs8, a_values, a_data, y_data, cov_inv, om_fdl, "LCDM"))
sampler_L.run_mcmc(pos_L, runs, progress=True)

100%|██████████| 3000/3000 [16:25<00:00,  3.04it/s]


State([[0.22656375 0.83912274]
 [0.15438838 0.99517612]
 [0.28122493 0.7577234 ]
 [0.26791681 0.79071504]
 [0.15840868 0.9886603 ]
 [0.28055677 0.78479205]
 [0.17665309 0.99520787]
 [0.23880032 0.82891418]
 [0.25330542 0.78587408]
 [0.21008827 0.83002637]
 [0.25815904 0.80626402]
 [0.18833892 0.94903023]
 [0.30662182 0.76362725]
 [0.18208182 0.93733585]
 [0.20814763 0.86898907]
 [0.31038168 0.74452863]
 [0.22907422 0.83283251]
 [0.19106292 0.904249  ]
 [0.22253036 0.86330064]
 [0.17868272 0.98733735]
 [0.22677386 0.8186325 ]
 [0.26461964 0.82205681]
 [0.19589929 0.87955226]
 [0.25268232 0.82475281]
 [0.13487017 1.0567534 ]
 [0.29872962 0.77060211]
 [0.17203791 0.95229098]
 [0.25322497 0.79692705]
 [0.18317098 0.92835063]
 [0.26321267 0.80529044]], log_prob=[-12.00960968 -12.40170966 -12.92514357 -12.22642908 -12.24417225
 -12.41412307 -13.46453496 -11.92533562 -12.86187747 -14.13867564
 -12.08497341 -12.32329428 -13.00879611 -11.86603035 -11.99158599
 -12.92268659 -12.09337231 -11.9702

In [8]:
# sampler_G = emcee.EnsembleSampler(nwalkers, 2, lnprob, args=(fs8, a_values, a_data, y_data, cov_inv, om_fdl, "GCDM"))
# sampler_G.run_mcmc(pos_G, runs, progress=True)

In [9]:
samples_L = sampler_L.chain[:].reshape((-1, 2))
np.savez('samples_LCDM.npz', samples_L)

In [10]:
# samples_G = sampler_G.chain[:].reshape((-1, 2))
# np.savez('samples_GCDM.npz', samples_G)